# Implementando uma dimensão de data no postgres com python

In [1]:
import pandas as pd
import psycopg2
conn = psycopg2.connect(host='localhost',database='test', user='postgres', password='123456')

In [2]:
sql_generate_dim_data ="""

create table if not exists dim_data as SELECT
	datum AS DATE,
	EXTRACT(YEAR FROM datum) AS YEAR,
	EXTRACT(MONTH FROM datum) AS MONTH,
	-- Localized month name
	to_char(datum, 'TMMonth') AS MonthName,
	EXTRACT(DAY FROM datum) AS DAY,
	EXTRACT(doy FROM datum) AS DayOfYear,
	-- Localized weekday
	to_char(datum, 'TMDay') AS WeekdayName,
	-- ISO calendar week
	EXTRACT(week FROM datum) AS CalendarWeek,
	to_char(datum, 'dd. mm. yyyy') AS FormattedDate,
	'Q' || to_char(datum, 'Q') AS Quartal,
	to_char(datum, 'yyyy/"Q"Q') AS YearQuartal,
	to_char(datum, 'yyyy/mm') AS YearMonth,
	-- ISO calendar year and week
	to_char(datum, 'iyyy/IW') AS YearCalendarWeek,
	-- Weekend
	CASE WHEN EXTRACT(isodow FROM datum) IN (6, 7) THEN 'Weekend' ELSE 'Weekday' END AS Weekend,
	-- Fixed holidays 
        -- for America
        CASE WHEN to_char(datum, 'MMDD') IN ('0101', '0704', '1225', '1226')
		THEN 'Holiday' ELSE 'No holiday' END
		AS AmericanHoliday,
        -- for Austria
	CASE WHEN to_char(datum, 'MMDD') IN 
		('0101', '0106', '0501', '0815', '1101', '1208', '1225', '1226') 
		THEN 'Holiday' ELSE 'No holiday' END 
		AS AustrianHoliday,
        -- for Canada
        CASE WHEN to_char(datum, 'MMDD') IN ('0101', '0701', '1225', '1226')
		THEN 'Holiday' ELSE 'No holiday' END 
		AS CanadianHoliday,
	-- Some periods of the year, adjust for your organisation and country
	CASE WHEN to_char(datum, 'MMDD') BETWEEN '0701' AND '0831' THEN 'Summer break'
	     WHEN to_char(datum, 'MMDD') BETWEEN '1115' AND '1225' THEN 'Christmas season'
	     WHEN to_char(datum, 'MMDD') > '1225' OR to_char(datum, 'MMDD') <= '0106' THEN 'Winter break'
		ELSE 'Normal' END
		AS Period,
	-- ISO start and end of the week of this date
	datum + (1 - EXTRACT(isodow FROM datum))::INTEGER AS CWStart,
	datum + (7 - EXTRACT(isodow FROM datum))::INTEGER AS CWEnd,
	-- Start and end of the month of this date
	datum + (1 - EXTRACT(DAY FROM datum))::INTEGER AS MonthStart,
	(datum + (1 - EXTRACT(DAY FROM datum))::INTEGER + '1 month'::INTERVAL)::DATE - '1 day'::INTERVAL AS MonthEnd
FROM (
	-- There are 3 leap years in this range, so calculate 365 * 10 + 3 records
	SELECT '2000-01-01'::DATE + SEQUENCE.DAY AS datum
	FROM generate_series(0,3652) AS SEQUENCE(DAY)
	GROUP BY SEQUENCE.DAY
     ) DQ
ORDER BY 1
"""

In [3]:
cur = conn.cursor()
cur.execute(sql_generate_dim_data)
cur.execute("commit")
#or pretty view
res = pd.read_sql_query("select * from dim_data limit 10",conn)
res

,date,year,month,monthname,day,dayofyear,weekdayname,calendarweek,formatteddate,quartal,...,yearcalendarweek,weekend,americanholiday,austrianholiday,canadianholiday,period,cwstart,cwend,monthstart,monthend
0,2000-01-01,2000.0,1.0,Janeiro,1.0,1.0,Sábado,52.0,01. 01. 2000,Q1,...,1999/52,Weekend,Holiday,Holiday,Holiday,Winter break,1999-12-27,2000-01-02,2000-01-01,2000-01-31
1,2000-01-02,2000.0,1.0,Janeiro,2.0,2.0,Domingo,52.0,02. 01. 2000,Q1,...,1999/52,Weekend,No holiday,No holiday,No holiday,Winter break,1999-12-27,2000-01-02,2000-01-01,2000-01-31
2,2000-01-03,2000.0,1.0,Janeiro,3.0,3.0,Segunda,1.0,03. 01. 2000,Q1,...,2000/01,Weekday,No holiday,No holiday,No holiday,Winter break,2000-01-03,2000-01-09,2000-01-01,2000-01-31
3,2000-01-04,2000.0,1.0,Janeiro,4.0,4.0,Terça,1.0,04. 01. 2000,Q1,...,2000/01,Weekday,No holiday,No holiday,No holiday,Winter break,2000-01-03,2000-01-09,2000-01-01,2000-01-31
4,2000-01-05,2000.0,1.0,Janeiro,5.0,5.0,Quarta,1.0,05. 01. 2000,Q1,...,2000/01,Weekday,No holiday,No holiday,No holiday,Winter break,2000-01-03,2000-01-09,2000-01-01,2000-01-31
5,2000-01-06,2000.0,1.0,Janeiro,6.0,6.0,Quinta,1.0,06. 01. 2000,Q1,...,2000/01,Weekday,No holiday,Holiday,No holiday,Winter break,2000-01-03,2000-01-09,2000-01-01,2000-01-31
6,2000-01-07,2000.0,1.0,Janeiro,7.0,7.0,Sexta,1.0,07. 01. 2000,Q1,...,2000/01,Weekday,No holiday,No holiday,No holiday,Normal,2000-01-03,2000-01-09,2000-01-01,2000-01-31
7,2000-01-08,2000.0,1.0,Janeiro,8.0,8.0,Sábado,1.0,08. 01. 2000,Q1,...,2000/01,Weekend,No holiday,No holiday,No holiday,Normal,2000-01-03,2000-01-09,2000-01-01,2000-01-31
8,2000-01-09,2000.0,1.0,Janeiro,9.0,9.0,Domingo,1.0,09. 01. 2000,Q1,...,2000/01,Weekend,No holiday,No holiday,No holiday,Normal,2000-01-03,2000-01-09,2000-01-01,2000-01-31
9,2000-01-10,2000.0,1.0,Janeiro,10.0,10.0,Segunda,2.0,10. 01. 2000,Q1,...,2000/02,Weekday,No holiday,No holiday,No holiday,Normal,2000-01-10,2000-01-16,2000-01-01,2000-01-31


### Também podemos utilizar o cursor para varrer as informações 

In [4]:
cur.execute("select * from dim_data limit 10")
rows = cur.fetchall()
for row in rows:
    print(row)


(datetime.date(2000, 1, 1), 2000.0, 1.0, 'Janeiro', 1.0, 1.0, 'Sábado', 52.0, '01. 01. 2000', 'Q1', '2000/Q1', '2000/01', '1999/52', 'Weekend', 'Holiday', 'Holiday', 'Holiday', 'Winter break', datetime.date(1999, 12, 27), datetime.date(2000, 1, 2), datetime.date(2000, 1, 1), datetime.datetime(2000, 1, 31, 0, 0))
(datetime.date(2000, 1, 2), 2000.0, 1.0, 'Janeiro', 2.0, 2.0, 'Domingo', 52.0, '02. 01. 2000', 'Q1', '2000/Q1', '2000/01', '1999/52', 'Weekend', 'No holiday', 'No holiday', 'No holiday', 'Winter break', datetime.date(1999, 12, 27), datetime.date(2000, 1, 2), datetime.date(2000, 1, 1), datetime.datetime(2000, 1, 31, 0, 0))
(datetime.date(2000, 1, 3), 2000.0, 1.0, 'Janeiro', 3.0, 3.0, 'Segunda', 1.0, '03. 01. 2000', 'Q1', '2000/Q1', '2000/01', '2000/01', 'Weekday', 'No holiday', 'No holiday', 'No holiday', 'Winter break', datetime.date(2000, 1, 3), datetime.date(2000, 1, 9), datetime.date(2000, 1, 1), datetime.datetime(2000, 1, 31, 0, 0))
(datetime.date(2000, 1, 4), 2000.0, 1.0, 

## fechando conexão

In [5]:
conn.close()

Reference: https://wiki.postgresql.org/wiki/Date_and_Time_dimensions